In [1]:
import pandas as pd
from gurobipy import GRB, Model, quicksum

import utils.util as utils

In [2]:
cities_df = pd.read_csv('data/cities.csv')
cities_df

,Area,City,x,y,TB/s
0,1,Warszawa,100,65,-
1,1,Skierniewice,92,62,197
2,1,Lódz,85,58,317
3,1,Konin,68,64,236
4,1,Sieradz,72,55,178
5,1,Piotrków,89,51,139
6,1,Radom,102,50,288
7,2,Kielce,98,42,-
8,2,Czestochowa,79,40,278
9,2,Bytom,76,31,163


In [3]:
area_df = pd.read_csv('data/area.csv')
area_df['Include at least'] = area_df['Include at least'].apply(lambda x: [group.split(';') for group in x.split('|')] if pd.notna(x) else [])
area_df['Include at most'] = area_df['Include at most'].apply(lambda x: [group.split(';') for group in x.split('|')] if pd.notna(x) else [])
area_df

,Area,Minimum cities in main line,Starting city,Ending city,Include at least,Include at most
0,1,4,Warszawa,Radom,[[Konin]],[]
1,2,5,Kielce,Gliwice,[],"[[Bytom, Sosnowiec, Katowice], [Wodzisław Śląs..."
2,3,5,Opole,Jelenia Góra,"[[Kalisz], [Poznań, Zielona Góra, Leszno]]",[]


### Parameters

In [26]:
A = cities_df['Area'].unique().tolist()
C = {a: cities_df[cities_df['Area'] == a]['City'].tolist() for a in A}
print(C)

M = {a: area_df[area_df['Area'] == a]['Minimum cities in main line'].values[0] for a in A}
S = {a: area_df[area_df['Area'] == a]['Starting city'].values[0] for a in A}
E = {a: area_df[area_df['Area'] == a]['Ending city'].values[0] for a in A}

d = {(a, c1, c2): utils.euclidean((cities_df[cities_df['City'] == c1]['x'].values[0], cities_df[cities_df['City'] == c1]['y'].values[0]), (cities_df[cities_df['City'] == c2]['x'].values[0], cities_df[cities_df['City'] == c2]['y'].values[0])) for a in A for c1 in C[a] for c2 in C[a] if c1 != c2}

{1: ['Warszawa', 'Skierniewice', 'Lódz', 'Konin', 'Sieradz', 'Piotrków', 'Radom'], 2: ['Kielce', 'Czestochowa', 'Bytom', 'Gliwice', 'Sosnowiec', 'Kraków', 'Bielsko', 'Katowice', 'Jastrzębie-Zdrój', 'Wodzisław Śląski'], 3: ['Poznań', 'Zielona Góra', 'Leszno', 'Kalisz', 'Legnica', 'Wrocław', 'Jelenia Góra', 'Wałbrzych', 'Opole']}


In [5]:
model = Model('Main line optimization')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-29


### Variables

In [6]:
x = model.addVars([(a, c1, c2) for a in A for c1 in C[a] for c2 in C[a] if c1 != c2], vtype=GRB.BINARY, name='x')
y = model.addVars([(a, c) for a in A for c in C[a]], vtype=GRB.BINARY, name='y')

### Objective function

In [7]:
model.setObjective(quicksum(d[(a, c1, c2)]*x[a, c1, c2] for a in A for c1 in C[a] for c2 in C[a] if c1 != c2), GRB.MINIMIZE)

### Starting and ending city constraint

In [8]:
for a in A:
    model.addConstr(y[a, S[a]] == 1, name=f'Starting city {a}')
    model.addConstr(y[a, E[a]] == 1, name=f'Ending city {a}')

    model.addConstr(quicksum(x[a, S[a], c] for c in C[a] if c != S[a]) >= 1, name=f'Starting city {a} out')
    model.addConstr(quicksum(x[a, c, E[a]] for c in C[a] if c != E[a]) >= 1, name=f'Ending city {a} in')

    model.addConstr(quicksum(x[a, c, S[a]] for c in C[a] if c != S[a]) == 0, name=f'Starting city {a} in')
    model.addConstr(quicksum(x[a, E[a], c] for c in C[a] if c != E[a]) == 0, name=f'Ending city {a} out')

### Satisfaction constraint

In [9]:
model.addConstrs((quicksum((y[a, c] for c in C[a])) >= M[a] for a in A), name='Minimum cities in main line')

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### Local growth constraint, Decentralization constraint, German connection constraint

In [10]:
# for index, row in area_df.iterrows():
#     area = row['Area']
#     include_at_least = row['Include at least']

#     if include_at_least:
#         for group in include_at_least:
#             model.addConstr(quicksum(y[area, c] for c in group) >= 1, name=f'Include at least {area}')

In [11]:
# for index, row in area_df.iterrows():
#     area = row['Area']
#     include_at_most = row['Include at most']

#     if include_at_most:
#         for group in include_at_most:
#             model.addConstr(quicksum(y[area, c] for c in group) <= 1, name=f'Include at most {area}')

In [12]:
model.addConstr((y[1, "Konin"] + y[3, "Kalisz"]) >= 1, name='Konin-Kalisz')
model.addConstr((y[3, "Poznań"] + y[3, "Zielona Góra"] + y[3, "Leszno"]) >= 1, name='Poznań, Zielona & Góra-Leszno')

model.addConstr((y[2, "Bytom"] + y[2, "Sosnowiec"] + y[2, "Katowice"]) <= 1, name='Bytom, Sosnowiec & Katowice')
model.addConstr((y[2, "Wodzisław Śląski"] + y[2, "Jastrzębie-Zdrój"]) <= 1, name='Wodzisław Śląski & Jastrzębie-Zdrój')

<gurobi.Constr *Awaiting Model Update*>

### Connection constraint

In [13]:
for a in A:
    for c1 in C[a]:
        model.addConstr(quicksum(x[a,c2,c1] for c2 in C[a] if c2 != c1) <= y[a,c1])


# The sum of all connections in an area must be equal to number of cities that is inclued (y[a,c] == 1)
# for a in A:
#     model.addConstr(quicksum(x[a,c1,c2] for c1 in C[a] for c2 in C[a] if c1 != c2) == (quicksum(y[a,c] for c in C[a])-1))

### Flow balance constraint

In [14]:
for a in A:
    for c1 in C[a]:
        if c1 != S[a] and c1 != E[a]:
            model.addConstr(quicksum(x[a, c1, c2] for c2 in C[a] if c2 != c1) == y[a, c1], name=f'Flow constraint for {c1} in area {a}')

# for a in A:
#     for c1 in C[a]:
#         if c1 != S[a] and c1 != E[a]:
#             model.addConstr(
#                 quicksum(x[a, c1, c2] for c2 in C[a] if c2 != c1) == quicksum(x[a, c2, c1] for c2 in C[a] if c2 != c1), 
#                 name=f'Flow balance for {c1} in area {a}'
#             )


# for a in A:
#     for c1 in C[a]:
#         for c2 in C[a]:
#             if c1 != c2:
#                 model.addConstr(x[a, c1, c2] <= y[a, c1], name=f'Flow constraint for {c1} -> {c2} in area {a}')
#                 model.addConstr(x[a, c1, c2] <= y[a, c2], name=f'Flow constraint for {c2} -> {c1} in area {a}')

for a in A:
    for c1 in C[a]:
        for c2 in C[a]:
            if c1 != c2:
                model.addConstr(x[a, c1, c2] + x[a, c2, c1] <= 1, name=f'Anti-symmetry for {c1} and {c2} in area {a}')




## Optimize

In [15]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 275 rows, 230 columns and 950 nonzeros
Model fingerprint: 0x115f7e3c
Variable types: 0 continuous, 230 integer (230 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 213.2333267
Presolve removed 212 rows and 130 columns
Presolve time: 0.16s
Presolved: 63 rows, 100 columns, 310 nonzeros
Found heuristic solution: objective 203.3413037
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 1.281960e+02, 38 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [16]:
if model.status == GRB.OPTIMAL:
    # for a in A:
    #     for i in C[a]:
    #         if y[a, i].x > 0.5:
    #             print(f"Main line in area {a} passes through city {i}")
    for a in A:
        for i in C[a]:
            for j in C[a]:
                if i != j:
                    if x[a, i, j].x > 0.5:
                        print(f"Main line in area {a} passes through cities {i} and {j}")

Main line in area 1 passes through cities Warszawa and Skierniewice
Main line in area 1 passes through cities Skierniewice and Piotrków
Main line in area 1 passes through cities Piotrków and Radom
Main line in area 2 passes through cities Kielce and Kraków
Main line in area 2 passes through cities Kraków and Bielsko
Main line in area 2 passes through cities Bielsko and Jastrzębie-Zdrój
Main line in area 2 passes through cities Jastrzębie-Zdrój and Gliwice
Main line in area 3 passes through cities Leszno and Legnica
Main line in area 3 passes through cities Kalisz and Leszno
Main line in area 3 passes through cities Legnica and Jelenia Góra
Main line in area 3 passes through cities Opole and Kalisz


In [17]:
current_area = 1
current_city = S[current_area]

while current_area in A:
    if current_city == E[current_area]:
        print(f"Main line in area {current_area} ends in city {current_city}")
        current_area += 1
        if current_area not in A:
            break
        current_city = S[current_area]
        continue
    for city in C[current_area]:
        if city == current_city:
            continue
        if x[current_area, current_city, city].x > 0.5:
            next_city = city
            break
    print(f"Main line in area {current_area} passes through city {current_city} to city {next_city}")
    current_city = next_city


Main line in area 1 passes through city Warszawa to city Skierniewice
Main line in area 1 passes through city Skierniewice to city Piotrków
Main line in area 1 passes through city Piotrków to city Radom
Main line in area 1 ends in city Radom
Main line in area 2 passes through city Kielce to city Kraków
Main line in area 2 passes through city Kraków to city Bielsko
Main line in area 2 passes through city Bielsko to city Jastrzębie-Zdrój
Main line in area 2 passes through city Jastrzębie-Zdrój to city Gliwice
Main line in area 2 ends in city Gliwice
Main line in area 3 passes through city Opole to city Kalisz
Main line in area 3 passes through city Kalisz to city Leszno
Main line in area 3 passes through city Leszno to city Legnica
Main line in area 3 passes through city Legnica to city Jelenia Góra
Main line in area 3 ends in city Jelenia Góra


### Write to CSV

In [23]:
cities_mainline_df = cities_df.copy()
cities_mainline_df['in_mainline'] = 0
for a in A:
    for c in C[a]:
        if y[a, c].x > 0.5:
            cities_mainline_df.loc[cities_mainline_df['City'] == c, 'in_mainline'] = 1
cities_mainline_df.to_csv('data/cities_mainline.csv', index=False)